# VGG 19

- Paper: [2014.09.04] Very Deep Convolutional Networks For Large-Scale Image Recognition
- https://arxiv.org/abs/1409.1556

### [Package load]

In [2]:
import torch 
print('pytorch version: {}'.format(torch.__version__))

import torchvision
import torch.nn as nn
import numpy as np
from tqdm.notebook import tqdm
%matplotlib inline

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 2.2.2
pytorch version: 2.2.2
GPU 사용 가능 여부: False


### [Model: VGG19]

In [8]:
cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'] #16 + 3 =vgg19

class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        #self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        #x = self.avgpool(x)        VGG16 for original ver.
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def make_layers(cfg, batch_norm=False):
        layers = []
        in_channels = 3
        
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
                        
        return nn.Sequential(*layers)

In [9]:
vgg19 = VGG(VGG.make_layers(cfg),5,True).to(device)

In [10]:
vgg19

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [11]:
from torchsummary import summary
summary(vgg19, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [12]:
from torchinfo import summary
summary(vgg19, input_size=(1, 3, 224, 224), col_width=20, depth=100, row_settings=["depth", "var_names"], col_names=["input_size", "kernel_size", "output_size", "params_percent"])

Layer (type (var_name):depth-idx)        Input Shape          Kernel Shape         Output Shape         Param %
VGG (VGG)                                [1, 3, 224, 224]     --                   [1, 5]                    --
├─Sequential (features): 1-1             [1, 3, 224, 224]     --                   [1, 512, 7, 7]            --
│    └─Conv2d (0): 2-1                   [1, 3, 224, 224]     [3, 3]               [1, 64, 224, 224]      0.00%
│    └─ReLU (1): 2-2                     [1, 64, 224, 224]    --                   [1, 64, 224, 224]         --
│    └─Conv2d (2): 2-3                   [1, 64, 224, 224]    [3, 3]               [1, 64, 224, 224]      0.03%
│    └─ReLU (3): 2-4                     [1, 64, 224, 224]    --                   [1, 64, 224, 224]         --
│    └─MaxPool2d (4): 2-5                [1, 64, 224, 224]    2                    [1, 64, 112, 112]         --
│    └─Conv2d (5): 2-6                   [1, 64, 112, 112]    [3, 3]               [1, 128, 112, 112]   

### [Model: VGG19 Transfer learning]

In [23]:
class TransferVGG(nn.Module):
    def __init__(self, num_classes):
        super(TransferVGG, self).__init__()
        self.VGG = torchvision.models.vgg19(pretrained=True)
        self.VGG.classifier = nn.Sequential(
            nn.Linear(in_features=512*7*7, out_features=4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(in_features= 4096, out_features=num_classes)
        )

    def forward(self,x):
        return self.VGG(x)

In [24]:
vgg19_transfer = TransferVGG(num_classes=5).to(device)

In [25]:
vgg19_transfer

TransferVGG(
  (VGG): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16)

In [26]:
from torchsummary import summary
summary(vgg19_transfer, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [27]:
from torchinfo import summary
summary(vgg19_transfer, input_size=(1, 3, 224, 224), col_width=20, depth=100, row_settings=["depth", "var_names"], col_names=["input_size", "kernel_size", "output_size", "params_percent"])

Layer (type (var_name):depth-idx)             Input Shape          Kernel Shape         Output Shape         Param %
TransferVGG (TransferVGG)                     [1, 3, 224, 224]     --                   [1, 5]                    --
├─VGG (VGG): 1-1                              [1, 3, 224, 224]     --                   [1, 5]                    --
│    └─Sequential (features): 2-1             [1, 3, 224, 224]     --                   [1, 512, 7, 7]            --
│    │    └─Conv2d (0): 3-1                   [1, 3, 224, 224]     [3, 3]               [1, 64, 224, 224]      0.00%
│    │    └─ReLU (1): 3-2                     [1, 64, 224, 224]    --                   [1, 64, 224, 224]         --
│    │    └─Conv2d (2): 3-3                   [1, 64, 224, 224]    [3, 3]               [1, 64, 224, 224]      0.03%
│    │    └─ReLU (3): 3-4                     [1, 64, 224, 224]    --                   [1, 64, 224, 224]         --
│    │    └─MaxPool2d (4): 3-5                [1, 64, 224, 224] 

In [ ]:
count = 0
for name, param in vgg19_transfer.named_parameters():
    count += 1
    print(f"count:{count},",name, param.requires_grad)

count:1, VGG.features.0.weight True
count:2, VGG.features.0.bias True
count:3, VGG.features.2.weight True
count:4, VGG.features.2.bias True
count:5, VGG.features.5.weight True
count:6, VGG.features.5.bias True
count:7, VGG.features.7.weight True
count:8, VGG.features.7.bias True
count:9, VGG.features.10.weight True
count:10, VGG.features.10.bias True
count:11, VGG.features.12.weight True
count:12, VGG.features.12.bias True
count:13, VGG.features.14.weight True
count:14, VGG.features.14.bias True
count:15, VGG.features.16.weight True
count:16, VGG.features.16.bias True
count:17, VGG.features.19.weight True
count:18, VGG.features.19.bias True
count:19, VGG.features.21.weight True
count:20, VGG.features.21.bias True
count:21, VGG.features.23.weight True
count:22, VGG.features.23.bias True
count:23, VGG.features.25.weight True
count:24, VGG.features.25.bias True
count:25, VGG.features.28.weight True
count:26, VGG.features.28.bias True
count:27, VGG.features.30.weight True
count:28, VGG.fea

In [ ]:
# features의 3번째 maxpooling 다음부터 classifier까지 required_grad True로 바꾸기 (25 ~ 28)
count = 0
for param in vgg19_transfer.VGG.parameters():
    count += 1
    if count >= 25:
        param.requires_grad = True
    else:
        param.requires_grad = False

for name, param in vgg19_transfer.VGG.named_parameters():
    print(name, param.requires_grad)

features.0.weight False
features.0.bias False
features.2.weight False
features.2.bias False
features.5.weight False
features.5.bias False
features.7.weight False
features.7.bias False
features.10.weight False
features.10.bias False
features.12.weight False
features.12.bias False
features.14.weight False
features.14.bias False
features.16.weight False
features.16.bias False
features.19.weight False
features.19.bias False
features.21.weight False
features.21.bias False
features.23.weight False
features.23.bias False
features.25.weight False
features.25.bias False
features.28.weight True
features.28.bias True
features.30.weight True
features.30.bias True
features.32.weight True
features.32.bias True
features.34.weight True
features.34.bias True
classifier.0.weight True
classifier.0.bias True
classifier.3.weight True
classifier.3.bias True
classifier.6.weight True
classifier.6.bias True


In [ ]:
# 마지막 conv layer 4개를 initialization (conv 28, 30, 32, 34)
conv_index = 28
for name, param in vgg19_transfer.VGG.features.named_parameters():
    if name == str(conv_index)+'.weight':
        print('VGG19의 '+str(conv_index)+'번째 conv의 weight')
        nn.init.xavier_uniform_(param)
        print(name+'의 conv filter initialization setting 완료')
        print()
        conv_index += 2

# BatchNorm은 없어서 initialization 생략

VGG19의 28번째 conv의 weight
28.weight의 conv filter initialization setting 완료

VGG19의 30번째 conv의 weight
30.weight의 conv filter initialization setting 완료

VGG19의 32번째 conv의 weight
32.weight의 conv filter initialization setting 완료

VGG19의 34번째 conv의 weight
34.weight의 conv filter initialization setting 완료

